In [80]:
import os
import gmaps
import gmaps.datasets
import config
import pandas as pd
import numpy as np
import time
from pygeocoder import Geocoder
# Try to move it to environement variables.
gmaps.configure(api_key=config.key)

# Read Data

In [7]:
location = pd.read_csv('data/Gps.csv',names=["serial_no","id","time","long","lat"])
print(location.head())

   serial_no                id           time       long        lat
0    3435081  01ef64e4dec54025  1500138559805 -75.987328  42.108504
1    3435082  01ef64e4dec54025  1500138560805 -75.987328  42.108504
2    3435083  01ef64e4dec54025  1500138561807 -75.987328  42.108504
3    3435084  01ef64e4dec54025  1500138562810 -75.987328  42.108504
4    3435085  01ef64e4dec54025  1500138563813 -75.987328  42.108504


In [8]:
# GPS co-ordinates of the Garage.
start_lat  = 42.107844 
start_long = -75.987106

In [9]:
def pretty_time(raw):
    '''
    Converts time in epoch(milli seconds) to readable format.
    input: epoch time
    output: returns string formatted time
    '''
    return (time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(raw*.001)) )

print(pretty_time(location['time'][0]))

2017-07-15 13:09:19


### We have 4 different buses data in the same file.

In [10]:
bus_id = ( pd.DataFrame(location.groupby('id').count().reset_index()))
bus_id = bus_id['id']
print(bus_id)

0    01ef64e4dec54025
1    023d6dfce95b3997
2    024a0c2d305eb3cc
3    025391f6de955621
Name: id, dtype: object


## Filtering the Data.
* Removing rows which represent when the bus is parked in Garage

In [11]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [12]:
# Calculating distance from the garage.
location['distance'] = location.apply(lambda row: haversine(start_long,start_lat,row.long,row.lat),axis=1 )
print(location.shape)

(353020, 6)


In [55]:
# Removing locations at garage which are less than 0.2 km
location = location.drop( location[location.distance < 0.2].index )
location = location.drop(['serial_no'],axis=1)
print(location.shape)

(50890, 5)


### Categorizing different buses information.

In [57]:
bus1 = location[location['id']==bus_id[0]]
bus2 = location[location['id']==bus_id[1]]
bus3 = location[location['id']==bus_id[2]]
bus4 = location[location['id']==bus_id[3]]

# Analyzing Routes and Heatmaps

In [ ]:
# Only one bus was runing during this time.
route = bus4[ ['lat','long']].head(10000)
print(route)

In [62]:
fig = gmaps.figure()
route_layer = gmaps.symbol_layer(route, fill_color="green", stroke_color="red", scale=1)
fig.add_layer(route_layer)
fig

In [64]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(route)
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 13
fig.add_layer(heatmap_layer)
fig

In [65]:
print(bus4.head())

                     id           time       long        lat  distance
17771  025391f6de955621  1500122642932 -75.988420  42.106331  0.200184
17772  025391f6de955621  1500122643933 -75.988536  42.106281  0.210065
17773  025391f6de955621  1500122644933 -75.988661  42.106238  0.219886
17774  025391f6de955621  1500122645933 -75.988766  42.106183  0.229930
17775  025391f6de955621  1500122646933 -75.988859  42.106131  0.239166


In [73]:
print(haversine(bus4.lat.iloc[0],bus4.long.iloc[0],bus4.lat.iloc[20],bus4.long.iloc[20]))

0.2035773431035068


In [91]:
turn_markers=[]
temp_street=''
for index,row in bus4.iterrows():
    temp_lat = row['lat']
    temp_long = row['long']
    results = Geocoder.reverse_geocode(temp_lat, temp_long)
    if(results.route!= temp_street):
        turn_markers.append([temp_lat,temp_long])
    temp_street = results.route    
    i=500
    if(i<0):
        break
    i=i-1

GeocoderError: Error OVER_QUERY_LIMIT
Query: https://maps.google.com/maps/api/geocode/json?language=&region=&sensor=false&latlng=42.095621%2C-75.975528&bounds=

In [ ]:
from pygeocoder import Geocoder
results = Geocoder.reverse_geocode(df['latitude'][0], df['longitude'][0])